In [1]:
from sqlalchemy import *
from sqlalchemy import create_engine, ForeignKey
from sqlalchemy import Column, Date, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref
from sqlalchemy.orm import sessionmaker
from datetime import datetime

engine = create_engine('sqlite:///database.db', echo=True)
Base = declarative_base()
 
########################################################################
class Team(Base):
    """"""
    __tablename__ = "teams"
 
    id = Column(String, primary_key=True)
    display_name = Column(String)
    
    #----------------------------------------------------------------------
    def __init__(self, displayName, id):
        """"""
        self.display_name = displayName
        self.id = id

########################################################################
class Channel(Base):
    """"""
    __tablename__ = "Channels"
 
    id = Column(String, primary_key=True)
    display_name = Column(String)
    
    #----------------------------------------------------------------------
    def __init__(self, displayName, id):
        """"""
        self.display_name = displayName
        self.id = id

        
########################################################################
class Message(Base):
    """"""
    __tablename__ = "Messages"
 
    id = Column(String, primary_key=True)
    content = Column(String)
    created_datetime = Column(String)
    username = Column(String)
    user_id = Column(String)

    #----------------------------------------------------------------------
    def __init__(self, id, content, created_datetime, username, user_id):
        """"""
        self.id = id
        self.content = content
        self.created_datetime = created_datetime
        self.username = username
        self.user_id = user_id
        

########################################################################
class Reply(Base):
    """"""
    __tablename__ = "Replies"
 
    id = Column(String, primary_key=True)
    content = Column(String)
    created_datetime = Column(String)
    username = Column(String)
    user_id = Column(String)
    message_id = Column(String, ForeignKey('Messages.id', ondelete="CASCADE"))

    #----------------------------------------------------------------------
    def __init__(self, id, content, created_datetime, username, user_id, message_id):
        """"""
        self.id = id
        self.content = content
        self.created_datetime = created_datetime
        self.username = username
        self.user_id = user_id
        self.message_id = message_id



# create tables
Base.metadata.create_all(engine)

2020-12-17 10:11:06,628 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-17 10:11:06,634 INFO sqlalchemy.engine.base.Engine ()
2020-12-17 10:11:06,637 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-17 10:11:06,638 INFO sqlalchemy.engine.base.Engine ()
2020-12-17 10:11:06,644 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("teams")
2020-12-17 10:11:06,646 INFO sqlalchemy.engine.base.Engine ()
2020-12-17 10:11:06,648 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("teams")
2020-12-17 10:11:06,650 INFO sqlalchemy.engine.base.Engine ()
2020-12-17 10:11:06,654 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Channels")
2020-12-17 10:11:06,657 INFO sqlalchemy.engine.base.Engine ()
2020-12-17 10:11:06,659 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Channels")
2020-12-17 10:11:06,660 INFO sqlalchemy.engine.base.Engine ()
2020-12-17 10:11:06,

# Import des fonctions des exos précédents

In [2]:
%run ex01.ipynb
%run ex02.ipynb
%run ex03.ipynb

<Response [200]>
{"@odata.context":"https://graph.microsoft.com/beta/$metadata#directoryObjects","value":[{"@odata.type":"#microsoft.graph.group","id":"670f71a9-df13-4b92-80ae-eb7dc5099041","deletedDateTime":null,"classification":null,"createdDateTime":"2020-10-29T15:23:10Z","createdByAppId":"cc15fd57-2c6c-4117-a88c-83b1d56b4bbe","description":"[MRS] Coding","displayName":"[MRS] Dev&go","expirationDateTime":null,"groupTypes":["Unified"],"infoCatalogs":[],"isAssignableToRole":null,"isManagementRestricted":null,"mail":"MRSCoding@epitechfr.onmicrosoft.com","mailEnabled":true,"mailNickname":"MRSCoding","membershipRule":null,"membershipRuleProcessingState":null,"onPremisesDomainName":null,"onPremisesLastSyncDateTime":null,"onPremisesNetBiosName":null,"onPremisesSamAccountName":null,"onPremisesSecurityIdentifier":null,"onPremisesSyncEnabled":null,"preferredDataLocation":null,"preferredLanguage":null,"proxyAddresses":["SPO:SPO_62e205f8-7e84-474e-9c91-2020c1c6d4b2@SPO_901cb4ca-b862-4029-9306-e

# Import des données dans la database

In [3]:
# create a Session
Session = sessionmaker(bind=engine)
session = Session()


class DataImporter: # classe avec les méthodes d'import
    def __init__(self, teams_id, channels_id):
        self.teams_id = teams_id
        self.channels_id = channels_id
    
    def import_teams_into_database(self):
        teams = memberOf()
        for key in teams:
            #def __init__(self, displayName, id):
            team = Team(key, teams[key]) # création de l'objet team avec les valeurs récupérées par la fonction memberOf: key == displayName & teams[key] == id

            session.add(team) # insère les données contenues dans l'objet team dans la BDD / ORM
            session.commit()
        
        session.close()
    
    def import_channels_into_database(self):
        channels = getChannels()
        for key in channels:
            #def __init__(self, displayName, id):
            channel = Channel(key, channels[key])
            session.add(channel) # insère les données contenues dans l'objet channel dans la BDD
            session.commit()
        session.close()
    
    def import_messages_into_database(self):
        #print(self.teams_id, '\n', self.channels_id)
        messages = getMessages(self.teams_id, self.channels_id)
        #print(messages)
        
        for message in messages:
            #message constructor = def __init__(self, id, content, created_datetime, username, user_id):
            new_message = Message(message['id'], message['content'], message['createdDateTime'], message['userName'], message['userId'])

            session.add(new_message) # insère les données contenues dans l'objet new_message dans la BDD
            session.commit()
        
            for reply in message['replies']: #boucler sur le tableau de replies associé au message
#                 def __init__(self, id, content, created_datetime, username, user_id, message_id):

                #print(key)
                new_reply = Reply(reply['id'], reply['content'], reply['createdDateTime'], reply['userName'], reply['userId'], message['id'])
                
                session.add(new_reply) # insère les données contenues dans l'objet new_reply dans la BDD
                session.commit()
        
        session.close()

test = DataImporter(teamsId, channelId) # instancie l'objet
test.import_teams_into_database()  # appel à la méthode pour importer les teams
test.import_channels_into_database() # appel à la méthode pour importer les channels
test.import_messages_into_database() # appel à la méthode pour importer les messages + replies



{'General': '19:860d4f4a79cd4932a4a3daa29db8f784@thread.tacv2', 'C-DAT-100 - Python Data Structure': '19:36be2edc99144232a0819f8ec29a8055@thread.tacv2', 'C-DAT-100 - Python Data Structure Yet Again': '19:972cb074d0b94cc6b6fb1e2394ab1e69@thread.tacv2', 'C-DAT-100 - HTTP Requests': '19:d7f803282158482abebd31b264ae4c68@thread.tacv2', 'C-DAT-100 - Web Scraping': '19:090a3e6fd21f4a17ac8832e65d61a0c7@thread.tacv2', 'C-DAT-100 - HTTP Server': '19:dddfd7fdfefa4c909ab8afe560ce87bc@thread.tacv2', 'Groupe 03': '19:81dc314389154e1998411bf683932381@thread.tacv2'}
2020-12-17 10:13:12,590 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-17 10:13:12,593 INFO sqlalchemy.engine.base.Engine INSERT INTO teams (id, display_name) VALUES (?, ?)
2020-12-17 10:13:12,595 INFO sqlalchemy.engine.base.Engine ('19:860d4f4a79cd4932a4a3daa29db8f784@thread.tacv2', 'General')
2020-12-17 10:13:12,602 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-17 10:13:12,628 INFO sqlalchemy.engine.base.Engine BEGIN (im

2020-12-17 10:13:26,852 INFO sqlalchemy.engine.base.Engine ('1608137399977', '<div><a href="https://www.epitech.eu/fr/actualites-evenements/epitech-lance-son-cycle-de-conferences-ouvre-ta-tech/"><u></u></a><u><a href="https://w ... (819 characters truncated) ... ponsabilité de tous &amp; toutes.<br>\n<br>\nOn pourra notamment y entendre Thomas Solignac (promo 2015 / CEO Golem AI)<br>\n+ 6 autres experts</div>', '2020-12-16T16:49:59.977Z', 'Yoann Long', 'fe144cf4-ba41-4f1c-abfa-cd777aecb199', '1608134375757')
2020-12-17 10:13:26,859 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-17 10:13:26,889 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-17 10:13:26,891 INFO sqlalchemy.engine.base.Engine INSERT INTO "Messages" (id, content, created_datetime, username, user_id) VALUES (?, ?, ?, ?, ?)
2020-12-17 10:13:26,893 INFO sqlalchemy.engine.base.Engine ('1608134801088', '', '2020-12-16T16:06:41.088Z', 'Matthieu Gabas', '195d5507-ef82-48fe-b46b-499f4500413d')
2020-12-17 10:13:26,8

2020-12-17 10:13:27,177 INFO sqlalchemy.engine.base.Engine ('1607970076742', 'Oui il n’y a qu’une seule organisation mais dedans est ce que tu vois un repository qui s’appellerait “py_01” ?', '2020-12-14T18:21:16.742Z', 'Yoann Long', 'fe144cf4-ba41-4f1c-abfa-cd777aecb199', '1607969652764')
2020-12-17 10:13:27,182 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-17 10:13:27,197 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-17 10:13:27,199 INFO sqlalchemy.engine.base.Engine INSERT INTO "Replies" (id, content, created_datetime, username, user_id, message_id) VALUES (?, ?, ?, ?, ?, ?)
2020-12-17 10:13:27,200 INFO sqlalchemy.engine.base.Engine ('1607970020751', "J'ai qu'EpitechIT2020 dans mes organisations", '2020-12-14T18:20:20.751Z', 'Mohammed Toualbia', '0b16d9de-b6a4-47b2-bacb-c2f83ea125ed', '1607969652764')
2020-12-17 10:13:27,205 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-17 10:13:27,220 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-17 10:13:27,221 

2020-12-17 10:13:27,512 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-17 10:13:27,526 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-17 10:13:27,528 INFO sqlalchemy.engine.base.Engine INSERT INTO "Replies" (id, content, created_datetime, username, user_id, message_id) VALUES (?, ?, ?, ?, ?, ?)
2020-12-17 10:13:27,529 INFO sqlalchemy.engine.base.Engine ('1607959445770', "bah justement a priori mon problème vient de la mais je n'arrive pas a comprendre le chemin que je dois lui spécifié... Mon fichier json est dans le  ... (58 characters truncated) ... ommer simplement, de l'appeler avec un ./ devant pour lui dire que c'est dans le même dossier ou meme de lui donner le chemin absolue rien ne marche.", '2020-12-14T15:24:05.77Z', 'Benoit Spriet', '84b2934f-1f59-49e7-ad12-08f04b66b7b6', '1607958734957')
2020-12-17 10:13:27,535 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-17 10:13:27,552 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-17 10:13:27,554 INFO sq

2020-12-17 10:13:27,832 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-17 10:13:27,850 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-17 10:13:27,852 INFO sqlalchemy.engine.base.Engine INSERT INTO "Replies" (id, content, created_datetime, username, user_id, message_id) VALUES (?, ?, ?, ?, ?, ?)
2020-12-17 10:13:27,853 INFO sqlalchemy.engine.base.Engine ('1607957372613', '<div><div>Je serai avec <at id="0">Yann Fontaine</at>&nbsp;<span class="animated-emoticon-20-smile" title="Smiley" type="(smile)"><img itemid="smile" ... (49 characters truncated) ... moji" src="https://statics.teams.cdn.office.net/evergreen-assets/skype/v2/smile/20.png" alt="🙂" style="width:20px; height:20px"></span></div>\n</div>', '2020-12-14T14:49:32.613Z', 'David Ta', 'd98a5980-2ceb-4dab-a6e6-15fd5e7515ec', '1607957320888')
2020-12-17 10:13:27,856 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-17 10:13:27,873 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-17 10:13:27,875 INFO sqlalc

2020-12-17 10:13:28,179 INFO sqlalchemy.engine.base.Engine ('1607956231083', 'le planning a bien changé mais je suis toujours en code & go et à Lyon cette fois ci pour la ville.<attachment id="40cdff4a-70a0-439b-be60-00e4ec060529"></attachment>', '2020-12-14T14:30:31.083Z', 'Mohammed Toualbia', '0b16d9de-b6a4-47b2-bacb-c2f83ea125ed', '1607956100081')
2020-12-17 10:13:28,183 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-17 10:13:28,198 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-17 10:13:28,200 INFO sqlalchemy.engine.base.Engine INSERT INTO "Replies" (id, content, created_datetime, username, user_id, message_id) VALUES (?, ?, ?, ?, ?, ?)
2020-12-17 10:13:28,201 INFO sqlalchemy.engine.base.Engine ('1607956229057', 'pensez à vous inscrire à tout les "code review" (les briques jaunes)\xa0', '2020-12-14T14:30:29.057Z', 'Yoann Long', 'fe144cf4-ba41-4f1c-abfa-cd777aecb199', '1607956100081')
2020-12-17 10:13:28,204 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-17 10:13:

2020-12-17 10:13:28,478 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-17 10:13:28,494 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-17 10:13:28,496 INFO sqlalchemy.engine.base.Engine INSERT INTO "Messages" (id, content, created_datetime, username, user_id) VALUES (?, ?, ?, ?, ?)
2020-12-17 10:13:28,497 INFO sqlalchemy.engine.base.Engine ('1607936629053', 'Y a t il eu un enregistrement de fait pour la réunion de ce matin?', '2020-12-14T09:03:49.053Z', 'Mohammed Toualbia', '0b16d9de-b6a4-47b2-bacb-c2f83ea125ed')
2020-12-17 10:13:28,500 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-17 10:13:28,517 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-17 10:13:28,518 INFO sqlalchemy.engine.base.Engine INSERT INTO "Replies" (id, content, created_datetime, username, user_id, message_id) VALUES (?, ?, ?, ?, ?, ?)
2020-12-17 10:13:28,519 INFO sqlalchemy.engine.base.Engine ('1607936893324', '<div>Oui j’ai essayé de me reconnecter avec teams, impossible depuis le pc <